<a href="https://colab.research.google.com/github/Avipsa1/UPPP275-Notebooks/blob/main/Exploratory_Spatial_Data_Analysis_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Creating spatial weights matrix is an important aspect of examining spatial relationships across different spatial units or geometries. It defines how they're spatially connected to one another. These weights can take on many different forms. Based on our specific theoretical needs we can choose from the following:

* Rook contiguity
* Queen contiguity
* K-nearest neighbors
* Distance band

Using rook contiguity, two spatial units must share an edge of their boundaries to be considered neighbors. 



Let us install and import all necessary packages before we begin our analysis

In [ ]:
!pip install geopandas
!pip install pysal
!pip install seaborn

In [ ]:
import pandas as pd
import geopandas as gpd
import pysal as ps
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import stats

# load CA tracts, display shape
tracts_ca = gpd.read_file('tl_2017_06_tract.shp')
tracts_ca = tracts_ca.set_index('GEOID')
tracts_ca.shape

In [ ]:
# retain LA county only (and drop channel island tracts)
tracts_ca = tracts_ca[tracts_ca['COUNTYFP']=='037'].drop(index=['06037599100', '06037599000'])
tracts_ca.shape

In [ ]:
# project spatial geometries to a meter-based projection for SoCal
crs = '+proj=utm +zone=11 +ellps=WGS84 +datum=WGS84 +units=m +no_defs'
tracts_ca = tracts_ca.to_crs(crs)

In [ ]:
# load CA tract-level census variables
df_census = pd.read_csv('census_tracts_data_ca.csv', dtype={'GEOID10':str}).set_index('GEOID10')
df_census.shape

In [ ]:
# merge tract geometries with census variables
tracts = tracts_ca.merge(df_census, left_index=True, right_index=True, how='left')
tracts.shape

In [ ]:
# calculate pop density in persons per sq km
# turn any infinities into nulls
tracts['pop_density'] = tracts['total_pop'] / (tracts['ALAND'] / 1e6)
tracts = tracts.replace([np.inf, -np.inf], np.nan)

In [ ]:
tracts.head()

In [ ]:
tracts.columns

Now lets calculate the rook contiguity matrix from the data

In [ ]:
# get the tract labels (GEOIDs) and pick one (arbitrarily) to work with throughout
labels = tracts.index.tolist()
label = labels[210]
label

In [ ]:
%%time
import libpysal 
# calculate rook spatial weights
w_rook = libpysal.weights.Rook.from_dataframe(tracts, ids=labels, id_order=labels)

In [ ]:
# find the neighbors of some tract
# this is a raw contiguity matrix, so weights are binary 1s and 0s meaning neighbor/not
w_rook[label]

In [ ]:
w_rook.cardinalities['06037900701']

In [ ]:
w_rook.cardinalities['06037900705']

Using queen contiguity, two spatial units need only share a vertex (a single point) of their boundaries to be considered neighbors.

In [ ]:
%%time
# calculate queen spatial weights
w_queen = libpysal.weights.Queen.from_dataframe(tracts, ids=labels, id_order=labels)

In [ ]:
# find the neighbors of some tract
# this is a raw contiguity matrix, so weights are binary 1s and 0s meaning neighbor/not
w_queen[label]

In [ ]:
# how many neighbors does this tract have?
w_queen.cardinalities['06037900701']

In [ ]:
w_queen.cardinalities['06037900704']

In [ ]:
# convert cardinalities to series and describe data
pd.Series(w_queen.cardinalities).describe()

In [ ]:
# min number of neighbors
w_queen.min_neighbors

In [ ]:
# max number of neighbors
w_queen.max_neighbors

In [ ]:
# islands are observations with no neighbors, disconnected in space (can cause modeling problems)
w_queen.islands

Plot a census tract of interest, along with its neighbors:

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
tracts.plot(ax=ax, facecolor='#6f6a66', edgecolor='w', linewidth=0.5)

# plot some tract of interest in blue
tract = tracts.loc[[label]]
tract.plot(ax=ax, facecolor='#1f03d0', edgecolor='w', linewidth=2)

# plot the neighbors in pink
neighbors = tracts.loc[w_queen[label]]
neighbors.plot(ax=ax, facecolor='#c133cc', edgecolor='w', linewidth=2)

# zoom to area of interest
xmin, ymin, xmax, ymax = neighbors.unary_union.bounds
ax.axis('equal')
ax.set_xlim(xmin-100, xmax+100)  # +/- 100 meters
ax.set_ylim(ymin, ymax)

ax.set_title('Neighbors of tract {}'.format(label))
_ = ax.axis('off')

In [ ]:
%%time
# draw a queen-contiguity graph of the tracts
fig, ax = plt.subplots(figsize=(12, 12), facecolor='#1ff1e1')
tracts.plot(ax=ax, facecolor='gray', edgecolor='k', linewidth=0.3)

# extract centroids of tract and its neighbors, then draw lines between them
for tract, neighbors in w_queen:
    tract_centroid = tracts.loc[tract, 'geometry'].centroid
    for neighbor_centroid in tracts.loc[neighbors, 'geometry'].centroid:
        Xs = [tract_centroid.x, neighbor_centroid.x]
        Ys = [tract_centroid.y, neighbor_centroid.y]
        ax.plot(Xs, Ys, color='b', linewidth=0.3)
_ = ax.axis('off')

Find the k-nearest neighbors of each tract, by centroid.

In [ ]:
%%time
# k-nearest neighbors finds the closest k tract centroids to each tract centroid
w_knn = libpysal.weights.KNN.from_dataframe(tracts, k=6)

In [ ]:
# they all have exactly k neighbors
w_knn.neighbors[label]

For distance-band contiguity matrix, other tracts are considered neighbors of some tract if they are within a given threshold distance from the centroid of one tract. Distance band weights can be specified to take on continuous values rather than binary (1s and 0s), with these values being the inverse distance between each pair of "neighboring" units.

There are two ways to define distance decay:

* linear distance-decay exponent is -1 : 1/d
* gravity model distance-decay exponent is -2 : 1/d^2

In [ ]:
# calculate maximum nearest neighbor distance so each unit is assured of >=1 neighbor
x = tracts.centroid.x
y = tracts.centroid.y
coords = np.array([x, y]).T
threshold = libpysal.weights.min_threshold_distance(coords)
threshold

In [ ]:
%%time
# calculate linear decay continuous weights
w_dist = libpysal.weights.distance.DistanceBand.from_dataframe(tracts,
                                                             threshold=threshold,
                                                             binary=False,
                                                             alpha=-1)

In [ ]:
# how many distance-band neighbors does our tract have?
len(w_dist.neighbors[label])

In [ ]:
# map the neighbors, colored by weight from nearest to furthest
fig, ax = plt.subplots(figsize=(6, 6))
tracts.plot(ax=ax, facecolor='#333333', edgecolor='gray', linewidth=0.1)

# get the tract of interest and its neighbors/weights
tract = tracts.loc[[label]]
weights = pd.Series(w_dist[label])
neighbors = tracts.loc[weights.index, ['geometry']]
neighbors['weights_scaled'] = weights

# plot the tract's neighbors in blues by weight
neighbors.plot(ax=ax,
               column='weights_scaled',
               cmap='Blues_r',
               edgecolor='gray',
               linewidth=0.3,
               scheme='NaturalBreaks')

# plot the tract of interest in red
tract.plot(ax=ax, facecolor='r', edgecolor='r', linewidth=0.1)

# zoom to area of interest
xmin, ymin, xmax, ymax = neighbors.unary_union.bounds
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_title('Neighbors of tract {}'.format(label))
_ = ax.axis('off')

## Now it's your turn
* Recompute the distance-based spatial weights with a gravity decay
* Try to think and describe how and why this impacts the number of neighbors and the map above? 
